In [48]:
import pyodbc
import pandas as pd
import sqlite3 as sqlite
import sqlite3
from sqlite3 import Error
import seaborn as sns

#1. Retorna a última consulta em Banco de dados (local)
try:

    sqliteConnection = sqlite3.connect(r'C:\Users\lucaspb\Documents\Dev\Indicadores_camed\Database\Indicador_GasesDB.db')
    cursor = sqliteConnection.cursor()
    print("Conexão bem sucedida ao SQLite")
    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    print("SQLite version is: ", record)

    sql_guia_eventosQuery = pd.read_sql("select * from Gases_diarias", con=sqliteConnection)
    
    cursor.close()

except sqlite3.Error as error:
    print("Erro ao tentar conectar SQLite: ", error)

finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("Conexão SQLite fechada")
        
        sql_guia_eventosQuery['DATAATENDIMENTO2'] = pd.to_datetime(sql_guia_eventosQuery['DATAATENDIMENTO'].str.slice(start=0, stop=10))
        print("A data base do relatório é (último atendimento): ")
        print(sql_guia_eventosQuery['DATAATENDIMENTO2'].max())
        #print(sql_guia_eventosQuery['DATAATENDIMENTO2'])

#2. classifica os eventos em oxigênio, diária, UTI e etc

#Cria série única de códigos dos eventos (Não funciona ao fazer merge com a base completa):
df_classificaEventos = pd.DataFrame(data=None)
df_classificaEventos['ESTRUTURA'] = pd.Series(sql_guia_eventosQuery['ESTRUTURA'].unique())
df2 = pd.merge(df_classificaEventos, sql_guia_eventosQuery, left_on='ESTRUTURA', right_on='ESTRUTURA')

#Strings que Classificam os eventos
searchStringAcompanhante = 'acompanhante'
searchStringTrue = 'True'
searchStringFlase = 'False'
searchStringDiariainternacao = ['diaria', 'diária']
searchStringDia = ['dia', 'diá']
searchStringUTI = ' uti '
searchStringHora = 'hora'
searchStringMinuto = 'minuto'
searchStringEventosgases = ['6.00.34.335',
'6.00.28.378',
'6.00.34.122',
'6.00.28.475',
'6.00.28.467',
'6.00.28.599',
'6.00.28.335',
'6.00.28.394',
'6.00.11.017',
'6.00.04.762',
'6.00.04.614',
'6.00.28.432',
'6.00.28.440',
'6.00.04.746',
'6.00.10.770',
'6.00.34.343',
'6.00.04.592',
'6.00.28.343',
'6.00.28.491',
'6.00.11.009',
'6.00.10.851',
'6.00.28.459',
'6.00.02.824',
'6.00.04.754',
'6.00.04.606',
'6.00.04.460',
'6.00.34.157',
'6.00.04.894',
'6.00.08.016',
'6.00.11.246',
'6.00.11.254',
'6.00.11.092',
'6.00.34.130',
'6.00.11.106',
'6.00.05.866',
'6.00.28.351',
'6.00.06.013',
'6.00.10.045',
'6.00.27.738',
'6.00.28.564',
'0.82.04.1067',
'6.00.11.084',
'0.80.08.2033',
'6.00.28.521',
'6.00.04.444',
'6.00.08.318',
'6.00.02.662',
'6.00.10.053',
'6.00.34.327',
'6.00.04.142',
'6.00.04.312',
'0.80.08.1240',
'6.00.34.238',
'6.00.04.452',
'6.00.04.010',
'6.00.04.169',
'6.00.27.762',
'6.00.25.484',
'6.00.06.021',
'6.00.28.513',
'6.00.10.452',
'6.00.34.106',
'6.00.11.327',
'6.00.34.483',
'6.00.26.391',
'6.00.04.371',
'0.82.02.1031',
'6.00.02.816',
'6.00.05.874',
'6.00.34.300',
'6.00.11.335',
'6.00.34.203',
'6.00.32.120',
'6.00.03.847',
'0.80.08.1940',
'6.00.05.882',
'6.00.10.207',
'6.00.10.444',
'6.00.34.211',
'6.00.31.808',
'6.00.32.081',
'0.80.08.1304',
'0.80.08.1630',
'6.00.06.048',
'6.00.28.483',
'6.00.34.165',
'6.00.26.332',
'6.00.01.860',
'6.00.02.263',
'6.00.03.065',
'6.00.04.002',
'6.00.04.150',
'6.00.04.290',
'6.00.04.304',
'0.80.08.1290',
'0.80.08.1690',
'0.80.08.1711',
'0.80.08.1908',
'0.80.08.2165',
'6.00.05.890',
'6.00.06.790',
'6.00.09.977',
'6.00.08.326',
'6.00.08.334',
'6.00.09.063',
'6.00.34.297',
'6.00.10.843',
'6.00.14.911',
'6.00.27.681']

#Padroniza os arrays de palavras chave para a função .contains
searchStringDiariainternacao_pattern = '|'.join(searchStringDiariainternacao)
searchStringDia_pattern = '|'.join(searchStringDia)
searchStringEventosgases_pattern = '|'.join(searchStringEventosgases)

#Padroniza a descrição para minusculo
df2['Lower tdescricao'] = df2['tdescricao'].str.lower()

#3. Cria o indicador de Gases x Horas
df2['Flag diarias'] = df2['Lower tdescricao'].str.contains(searchStringDiariainternacao_pattern)
df2['Flag gases'] = df2['ESTRUTURA'].str.contains(searchStringEventosgases_pattern)

#Atribui o valor de consumo por hora
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringDia_pattern) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 24
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringHora) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 1
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringMinuto) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 1/60

#Classifica em evento válido (gases ou diárias)
df2.loc[(df2['Flag gases']  == True) | (df2['Flag diarias'] == True), 'Evento valido'] = 1

#Consumo em horas de gases e diárias, por evento
df2.loc[df2['Flag diarias']  == True, 'Diárias (horas)'] = 24*df2['qtdpagto']
df2.loc[(df2['Flag gases']  == True), 'Consumo de gases (horas)'] = df2['Unidade gases']*df2['qtdpagto']

#Cria pivottable para comparação do total de horas entre gases e diárias

#adicionado esse trecho: Cria duas Pivot tables, uma com total de horas por evento de gases e outra com total de internação por guia
df2_pivot_Gases = df2.pivot_table(values = ['Consumo de gases (horas)'], index = ['GUIA', 'PRESTADOR', 'Lower tdescricao'], aggfunc = 'sum')
df2_pivot_Diárias = df2.pivot_table(values = ['Diárias (horas)'], index = ['GUIA', 'PRESTADOR'], aggfunc = 'sum')

#cria um join entre as pivot tables acima para relacionar por evento de gases, se a guia correspondente possui horas a maior
df2_join_Gases_Diarias = df2_pivot_Gases.merge(df2_pivot_Diárias, on=['GUIA'])

#Compara os consumos de horas
df2_join_Gases_Diarias.loc[(df2_join_Gases_Diarias['Consumo de gases (horas)'] > df2_join_Gases_Diarias['Diárias (horas)']), 'Consumo maior'] = "Sim"
df2_join_Gases_Diarias.loc[(df2_join_Gases_Diarias['Consumo de gases (horas)'] < df2_join_Gases_Diarias['Diárias (horas)']), 'Consumo maior'] = "Não"
df2_join_Gases_Diarias.loc[(df2_join_Gases_Diarias['Consumo de gases (horas)'] == df2_join_Gases_Diarias['Diárias (horas)']), 'Consumo maior'] = "Igual"
df2_join_Gases_Diarias['Diferença em horas'] = df2_join_Gases_Diarias['Consumo de gases (horas)'] - df2_join_Gases_Diarias['Diárias (horas)'] 

# 4.1. Cria indicadores de conflitos com diárias de acompanhantes

#Padroniza a descrição para posterior busca
sql_guia_eventosQuery['descricao_limpa'] = sql_guia_eventosQuery['tdescricao'].str.lower().replace(regex=['-','/'], value=" ")

#Classifica os eventos de acompanhantes, UTI e não UTI
sql_guia_eventosQuery.loc[(sql_guia_eventosQuery['descricao_limpa'].str.contains(searchStringDiariainternacao_pattern) == True) & (sql_guia_eventosQuery['descricao_limpa'].str.contains(searchStringUTI) != True), 'Flag_nao_UTI'] = 1*sql_guia_eventosQuery['qtdpagto']
sql_guia_eventosQuery.loc[(sql_guia_eventosQuery['descricao_limpa'].str.contains(searchStringDia_pattern) == True) & (sql_guia_eventosQuery['descricao_limpa'].str.contains(searchStringUTI)== True), 'Flag_UTI'] = 1*sql_guia_eventosQuery['qtdpagto']
sql_guia_eventosQuery.loc[(sql_guia_eventosQuery['descricao_limpa'].str.contains(searchStringAcompanhante) == True), 'Flag_Acompanhante'] = 1*sql_guia_eventosQuery['qtdpagto']

#Cria Pivottable acompanhante 1
df_pivot_acompanhante = sql_guia_eventosQuery.pivot_table(values = ['Flag_Acompanhante'], index = ['GUIA'], aggfunc = 'sum')
df_pivot_acompanhante = df_pivot_acompanhante.loc[(df_pivot_acompanhante['Flag_Acompanhante'] > 0)]

#Cria Pivottable UTI 1
df_pivot_UTI = sql_guia_eventosQuery.pivot_table(values = ['Flag_UTI'], index = ['GUIA'], aggfunc = 'sum')                                                   
df_pivot_UTI = df_pivot_UTI.loc[(df_pivot_UTI['Flag_UTI'] > 0 )]                                                  

#Cria Pivottable Não UTI 1
df_pivot_Nao_UTI = sql_guia_eventosQuery.pivot_table(values = ['Flag_nao_UTI'], index = ['GUIA'], aggfunc = 'sum')                                                   
df_pivot_Nao_UTI = df_pivot_Nao_UTI.loc[(df_pivot_Nao_UTI['Flag_nao_UTI'] > 0 )]

#Merge das três pivots acima
df_join_UTI_acompanhantes_diarias = df_pivot_acompanhante.merge(df_pivot_UTI, how='left', on='GUIA')
df_join_UTI_acompanhantes_diarias = df_join_UTI_acompanhantes_diarias.merge(df_pivot_Nao_UTI, how='left', on='GUIA')

#compara se existe valor de acompanhante sem diária de Não UTI
df_join_UTI_acompanhantes_diarias.loc[(df_join_UTI_acompanhantes_diarias['Flag_Acompanhante']>0) & (df_join_UTI_acompanhantes_diarias['Flag_nao_UTI']==0), 'Compara-Diárias de acompanhante sem internação']='Sim'
#Compara se existe diária de acompanhante a mais que internação
df_join_UTI_acompanhantes_diarias.loc[(df_join_UTI_acompanhantes_diarias['Flag_Acompanhante'] > df_join_UTI_acompanhantes_diarias['Flag_nao_UTI']), 'Compara-Diárias de acompanhante maior que internação normal']='Sim'

#Gera os reports
#df_pivot_acompanhante.to_csv(r'C:\Users\lucaspb\Documents\Dev\Indicadores_camed\Testes\Testes - Conflitos acompanhantes\df_acompanhates', encoding='UTF-8')
#df_pivot_UTI.to_csv(r'C:\Users\lucaspb\Documents\Dev\Indicadores_camed\Testes\Testes - Conflitos acompanhantes\df_UTI', encoding='UTF-8')
#df_pivot_Nao_UTI.to_csv(r'C:\Users\lucaspb\Documents\Dev\Indicadores_camed\Testes\Testes - Conflitos acompanhantes\df_nao_UTI', encoding='UTF-8')
df_join_UTI_acompanhantes_diarias.to_csv(r'C:\Users\lucaspb\Documents\Dev\Indicadores_camed\Testes\Testes - Conflitos acompanhantes\df_nao_UTI', encoding='UTF-8')


Conexão bem sucedida ao SQLite
SQLite version is:  [('3.32.3',)]
Conexão SQLite fechada
A data base do relatório é (último atendimento): 
2020-09-03 00:00:00
